### Exercises:

Exercise: Suppose I have a box with a 6-sided die, an 8-sided die, and a 12-sided die. I choose one of the dice at random, roll it four times, and get 1, 3, 5, and 7. What is the probability that I chose the 8-sided die?

You can use the update_dice function or do the update yourself.

In [1]:
from empiricaldist import Pmf
import numpy as np

In [2]:
# Function to update the dice: Modified slightly (I like returning pmf object)
def update_dice(pmf, data):
    """Update pmf based on new data."""
    hypos = pmf.qs # get the sides of dice passed in
    likelihood = 1 / hypos # likelihood in dice is simply 1/N 
    impossible = (data > hypos) # boolean vector for when data pulled exceeds size of hypo
    likelihood[impossible] = 0 # need to mark likelihood as 0 ahead of time -> no longer possible to get this one
    pmf *= likelihood # update
    _ = pmf.normalize()
    return pmf

In [9]:
# key of interest
key = 8

# Establish hypotheses
hypos = [6, 8, 12]

# Build priors
prior = Pmf(1/3, hypos)

# Iterate through update for data received:
evidence = [1,3,5,7]

for roll in evidence:
    prior = update_dice(prior, roll)
    
# build posterior
posterior_dict = {q: p for q,p in zip(prior.qs, prior.ps)}
    
print(posterior_dict[key]) # 83.5%

0.8350515463917527


#### Exercise: Non-uniform prior

In the previous version of the dice problem, the prior probabilities are the same because the box contains one of each die. But suppose the box contains 1 die that is 4-sided, 2 dice that are 6-sided, 3 dice that are 8-sided, 4 dice that are 12-sided, and 5 dice that are 20-sided. I choose a die, roll it, and get a 7. What is the probability that I chose an 8-sided die?

Hint: To make the prior distribution, call Pmf with two parameters.

In [10]:
# key:
key = 8

# Establish hypotheses
hypos = [4, 6, 8, 12, 20]
probs = [1,2,3,4,5]

# Build priors
prior = Pmf(probs, hypos)

# Iterate through update for data received:
evidence = [7]

for roll in evidence:
    prior = update_dice(prior, roll)

# build posterior
posterior_dict = {q: p for q,p in zip(prior.qs, prior.ps)}
    
print(posterior_dict[key]) # 83.5%

0.391304347826087


### Exercise:

Suppose I have two sock drawers. One contains equal numbers of black and white socks. The other contains equal numbers of red, green, and blue socks. Suppose I choose a drawer at random, choose two socks at random, and I tell you that I got a matching pair. What is the probability that the socks are white?

For simplicity, let’s assume that there are so many socks in both drawers that removing one sock makes a negligible change to the proportions.

#### Approach:
- we have two hypotheses
- equal probability of these (prior)
- likelihood of these differs as follows:
    - For A (black and white) we have 2 ways to make a color-matching pair (BB, WW) out of 2^2 pairings (BB,WW,WB,BW)
        - When we pull a sock X, we have a 50% chance of then pulling another sock X
    - B has 3 total colors, which is going to increase the possible set:
        - When we pull a sock X, we have a 33% chance of then pulling another sock X
        
        
- From this we build our posterior, which can be used to answer the question of likelihood socks are white:

In [11]:
# Establish hypotheses
hypos = ['A', 'B']
probs = [1,1] # uniform

# Likelihood of pulling a pair
likelihood_pair = 1/2, 1/3

# Build priors
prior = Pmf(probs, hypos)

# Update posterior
posterior = prior * likelihood_pair
_ = posterior.normalize()
posterior 

,probs
A,0.6
B,0.4


In [13]:
# now we want to know they are white, which means we are pulling from A
# P(A & W) = P(A) * P(W|A)
0.6 * 0.5

0.3

#### Discussion Point: 

- Is my interpretation of the problem correct? Or did I luck into an answer here? 
    - We want to know likelihood of pulling a pair from A and that pair being white, which seems to align with the use of this joint probability: Pulling a Pair from A (A must since only white exists in A) and given you've pulled from A, likelihood it was white. 
    
- The overall problem could be reexpressed as:

$P(W|Pair) = \frac{P(White) * P(Pair | White)}{P(Pair)}$

Solved through the lens of only the drawer with A:
- P(White) = 0.5 * 0.5
- P(Pair | White) = 0.5
- P(Pair) -> need to break into our total law of prob here for likelihood of pair in general based on what we know about each box:
    - =  P(A) * P(Pair |A) +P(B) * P(Pair | B) = 0.5 * 0.5 + 0.5 * 0.333

In [16]:
(0.5**3)/ (0.25 + 0.5/3) # Same answer



0.30000000000000004

### Exercise:

Elvis Presley had a twin brother (who died at birth). What is the probability that Elvis was an identical twin?

Hint: In 1935, about 2/3 of twins were fraternal and 1/3 were identical. (This would be our prior)

In [17]:
hypo = ['Identical', 'Not Identical']
probs = [2/3, 1/3] # prior

# Likelihood for Elis
likelihood = 1/2, 1/2

# Build priors
prior = Pmf(probs, hypos)

# Update posterior
posterior = prior * likelihood_pair
_ = posterior.normalize()
posterior 


,probs
A,0.75
B,0.25
